# 17-05 구글 BERT의 다음 문장 예측(Next Sentence Prediction)

* 모든 BERT 실습은 Colab에서 진행한다고 가정합니다.
  
사전 학습된 한국어 BERT를 이용하여 다음 문장 예측을 실습해봅시다. 이번 실습을 위해서만이 아니라 앞으로 사전 학습된 BERT를 사용할 때는 transformers라는 패키지를 자주 사용하게 됩니다. 실습 환경에 transformers 패키지를 설치해둡시다.

In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.6 MB/s eta 0:00:00


In [3]:
import transformers
transformers.__version__

'4.25.1'

# 1. 모델 로드

transformers 패키지를 사용하여 모델과 토크나이저를 로드합니다. BERT는 이미 누군가가 학습해둔 모델을 사용하는 것이므로 우리가 사용하는 모델과 토크나이저는 항상 맵핑 관계여야 합니다.

In [4]:
from transformers import TFBertForNextSentencePrediction

TFBertForNextSentencePrediction.from_pretrained('BERT 모델 이름')을 넣으면 두 개의 문장이 이어지는 문장 관계인지 여부를 판단하는 BERT 구조를 로드합니다.

In [5]:
model = TFBertForNextSentencePrediction.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForNextSentencePrediction.

All the layers of TFBertForNextSentencePrediction were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


# 2. 토크나이저 로드

AutoTokenizer.from_pretrained('모델 이름')을 넣으면 해당 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드합니다.

In [6]:
from transformers import AutoTokenizer

In [7]:
import tensorflow as tf

In [8]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

# 3. 테스트

다음 문장 예측을 위해 문맥 상으로 실제로 이어지는 두 개의 문장을 준비합니다.

인덱스 0 : 실제 다음 문장  
인덱스 1 : 서로 상관없는 문장

In [9]:
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "pizza is eaten with the use of a knife and fork. In casual settings, however, it is cut into wedges to be eaten while held in the hand."

앞서 준비한 bert-base-uncased의 토크나이저를 사용하여 두 개의 문장을 정수 인코딩해봅시다.

In [10]:
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

토크나이저로 변환된 결과에서 input_ids를 통해 정수 인코딩 결과를 확인할 수 있습니다.

In [11]:
print(encoding['input_ids'])

tf.Tensor(
[[  101  1999  3304  1010 10733  2366  1999  5337 10906  1010  2107  2004
   2012  1037  4825  1010  2003  3591  4895 14540  6610  2094  1012   102
  10733  2003  8828  2007  1996  2224  1997  1037  5442  1998  9292  1012
   1999 10017 10906  1010  2174  1010  2009  2003  3013  2046 17632  2015
   2000  2022  8828  2096  2218  1999  1996  2192  1012   102]], shape=(1, 58), dtype=int32)


여기서 주의할 점은 여기서 101과 102는 특별 토큰이라는 점입니다. 실제로 해당 토크나이저의 [CLS] 토큰과 [SEP] 토큰의 번호를 출력해봅시다.

In [ ]:
print(tokenizer.cls_token, ':', tokenizer.cls_token_id)
print(tokenizer.sep_token, ':' , tokenizer.sep_token_id)

[CLS] : 101
[SEP] : 102


위의 정수 인코딩 결과를 다시 디코딩해보면 현재 입력의 구성을 확인할 수 있습니다.

In [12]:
print(tokenizer.decode(encoding['input_ids'][0]))

[CLS] in italy, pizza served in formal settings, such as at a restaurant, is presented unsliced. [SEP] pizza is eaten with the use of a knife and fork. in casual settings, however, it is cut into wedges to be eaten while held in the hand. [SEP]


BERT에서 두 개의 문장이 입력으로 들어갈 경우에는 맨 앞에는 [CLS] 토큰이 존재하고, 첫번째 문장이 끝나면 [SEP] 토큰, 그리고 두번째 문장이 종료되었을 때 다시 추가적으로 [SEP] 토큰이 추가됩니다. 토크나이저로 변환된 결과에서 token_type_ids를 통해서 문장을 구분하는 세그먼트 인코딩 결과를 확인할 수 있습니다.

In [ ]:
print(encoding['token_type_ids'])

tf.Tensor(
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]], shape=(1, 58), dtype=int32)


0이 연속적으로 등장하다가 어느 순간부터 1이 연속적으로 등장하는데, 이는 [CLS] 토큰의 위치부터 첫번째 문장이 끝나고나서 등장한 [SEP] 토큰까지의 위치에는 0이 등장하고, 다음 두번째 문장부터는 1이 등장하는 것입니다. token_type_ids에서는 0과 1로 두 개의 문장을 구분하고 있습니다.

##3.다음 문장 예측하기
이제 TFBertForNextSentencePrediction를 통해서 다음 문장을 예측해봅시다. 모델에 입력을 넣으면, 해당 모델은 소프트맥스 함수를 지나기 전의 값인 logits을 리턴합니다. 해당 값을 소프트맥스 함수를 통과시킨 후에 각 레이블에 대한 확률값을 출력해봅시다.

In [13]:
logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

In [14]:
logits

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 6.3957767, -6.3765645]], dtype=float32)>

In [15]:
softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print(probs)

tf.Tensor([[9.9999714e-01 2.8381912e-06]], shape=(1, 2), dtype=float32)


0번 인덱스에 대한 확률값이 1번 인덱스에 대한 확률값보다 훨씬 큽니다. 실질적으로 모델이 예측한 레이블은 0이라는 뜻입니다. 이제 두 개의 값 중 더 큰 값을 모델의 예측값으로 판단하도록 더 큰 확률값을 가진 인덱스를 리턴하도록 합니다.

In [ ]:
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [0]


최종 예측 레이블은 0입니다. 이는 BERT가 다음 문장 예측을 학습했을 당시에 실질적으로 이어지는 두 개의 문장의 레이블은 0. 이어지지 않는 두 개의 문장의 경우에는 레이블을 1로 두고서 이진 분류로 학습을 하였기 때문입니다. 이번에는 이어지지 않는 두 개의 문장으로 테스트해봅시다. 전체적인 과정은 이전과 같습니다.

In [16]:
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "The sky is blue due to the shorter wavelength of blue light."
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

In [17]:
logits

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-3.072948 ,  5.9056454]], dtype=float32)>

In [18]:
softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print(probs)

tf.Tensor([[1.2606411e-04 9.9987388e-01]], shape=(1, 2), dtype=float32)


In [19]:
tf.math.argmax(probs, axis=-1).numpy()

array([1])